In [1]:
# packages imported
import pandas as pd
import numpy as np
import mord
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import linear_model, preprocessing

# import data
# file_name 檔案路徑
file_name = "original_data/training_data.csv"
data = pd.read_csv(file_name, header = 0)
train = pd.DataFrame(data)
file_name_test = 'original_data/test_data.csv'
data = pd.read_csv(file_name_test, header = 0)
test = pd.DataFrame(data)

In [2]:
X_train = train['text']
Y_train = train['stars']
X_test = test['text']

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)

(7997,)
(7997,)
(2003,)


In [3]:
# freq matrix of X_train
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(X_train)
vectorizer = CountVectorizer()
vectorizer.fit(X_train)

train_text_freq = vectorizer.fit_transform(X_train)

transformer = TfidfTransformer(sublinear_tf=True, use_idf=True)
train_text_tfidf = transformer.fit_transform(train_text_freq)
train_word = vectorizer.get_feature_names()

# tfidf 矩陣
# print(total_text_tfidf.toarray())
# 總出現次數矩陣
# print(train_freq.toarray())
# features
# print(train_word)

In [4]:
train_text_tfidf.shape
len(Y_train)

7997

In [5]:
# freq matrix of X_test
tfidf_vectorizer_test = TfidfVectorizer()
tfidf_vectorizer_test.fit(X_test)
vectorizer_test = CountVectorizer()
vectorizer_test.fit(X_test)

test_text_freq = vectorizer.fit_transform(X_test)

transformer_test = TfidfTransformer(sublinear_tf=True, use_idf=True)
test_text_tfidf = transformer.fit_transform(test_text_freq)
test_word = vectorizer_test.get_feature_names()

In [6]:
test_text_tfidf.shape
len(test_word)

13880

In [7]:
# random forest feature importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

rf = RandomForestClassifier(criterion = 'entropy')
rf.fit(train_text_tfidf, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [8]:
n = 2000
sorted_features = sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), train_word), reverse=True)
first_n_features_of_train = [x[1] for x in sorted_features][0:n]
len(first_n_features_of_train)

2000

In [9]:
# chose the features from freq matrix of features selected by RF  (train)
df_train = pd.DataFrame(train_text_tfidf.toarray())
var_left_index = []

for i in range(len(first_n_features_of_train)):
    var_left_index.append(train_word.index(first_n_features_of_train[i]))
    
df_train_var_left = df_train.iloc[:,var_left_index]
df_train_var_left.columns = first_n_features_of_train

df_train_var_left.shape

(7997, 2000)

In [11]:
# chose the features from freq matrix of features selected by RF  (test)
from itertools import repeat

# 將test資料做轉換
# 同樣將test資料做斷詞 得到各單詞重複次數矩陣
# 對比該重複次數矩陣以及 R 篩選過後的單詞
# 只保留篩選後單詞的重複次數矩陣
# 若是重複次數矩陣中無該單詞  則新增該行 並填滿 0

X_colnames = first_n_features_of_train
# transformation of test_text_tfidf to DF should only be proceed "once"
# test_text_tfidf = pd.DataFrame(test_text_tfidf.toarray())


df_test_var_left = pd.DataFrame()

for i in range(len(X_colnames)):
    cur_name = X_colnames[i]
    
    if cur_name in test_word:
        cur_pos = test_word.index(cur_name)
        df_test_var_left[cur_name] = test_text_tfidf.iloc[:, cur_pos]
    else:
        df_test_var_left[cur_name] = pd.Series(list(repeat(0, 7997)))

In [12]:
df_test_var_left.shape

(2003, 2000)

In [156]:
# model fitting and prediction
import mord

clf = mord.LogisticAT(alpha = 1.)
clf.fit(df_train_var_left, Y_train)

#預測
test_pred = clf.predict(df_test_var_left)
review_id = test['review_id']
out = pd.DataFrame()
out[0] = review_id
out[1] = test_pred

out.to_csv('out.csv', header = False, index = False)

In [15]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'gamma': [0, 1, 2, 3, 4, 5, 6, 7],
}
# Create a based model
svr = svm.SVR()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = svr, param_grid = param_grid, 
                           scoring='mean_squared_error', cv = 5, verbose = 2)

grid_search.fit(df_train_var_left, Y_train)

Fitting 5 folds for each of 56 candidates, totalling 280 fits
[CV] C=0.001, gamma=0 ................................................


C:\Users\kami\Anaconda3\envs\data_sciense\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\Users\kami\Anaconda3\envs\data_sciense\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


[CV] ................................. C=0.001, gamma=0, total= 1.2min
[CV] C=0.001, gamma=0 ................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.2min remaining:    0.0s
C:\Users\kami\Anaconda3\envs\data_sciense\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\Users\kami\Anaconda3\envs\data_sciense\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


[CV] ................................. C=0.001, gamma=0, total= 1.2min
[CV] C=0.001, gamma=0 ................................................


...

[CV] .................................. C=1000, gamma=7, total= 1.6min
[CV] C=1000, gamma=7 .................................................


C:\Users\kami\Anaconda3\envs\data_sciense\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
C:\Users\kami\Anaconda3\envs\data_sciense\lib\site-packages\sklearn\metrics\scorer.py:100: DeprecationWarning: Scoring method mean_squared_error was renamed to neg_mean_squared_error in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


[CV] .................................. C=1000, gamma=7, total= 1.6min


[Parallel(n_jobs=1)]: Done 280 out of 280 | elapsed: 735.9min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'gamma': [0, 1, 2, 3, 4, 5, 6, 7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='mean_squared_error', verbose=2)

In [16]:
grid_search.grid_scores_,grid_search.best_params_, grid_search.best_score_

C:\Users\kami\Anaconda3\envs\data_sciense\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -1.57904, std: 0.05008, params: {'C': 0.001, 'gamma': 0},
  mean: -1.56390, std: 0.04986, params: {'C': 0.001, 'gamma': 1},
  mean: -1.56766, std: 0.04992, params: {'C': 0.001, 'gamma': 2},
  mean: -1.57224, std: 0.05001, params: {'C': 0.001, 'gamma': 3},
  mean: -1.57540, std: 0.05007, params: {'C': 0.001, 'gamma': 4},
  mean: -1.57728, std: 0.05009, params: {'C': 0.001, 'gamma': 5},
  mean: -1.57826, std: 0.05009, params: {'C': 0.001, 'gamma': 6},
  mean: -1.57878, std: 0.05010, params: {'C': 0.001, 'gamma': 7},
  mean: -1.57904, std: 0.05008, params: {'C': 0.01, 'gamma': 0},
  mean: -1.44680, std: 0.04554, params: {'C': 0.01, 'gamma': 1},
  mean: -1.47220, std: 0.04805, params: {'C': 0.01, 'gamma': 2},
  mean: -1.51358, std: 0.04920, params: {'C': 0.01, 'gamma': 3},
  mean: -1.54282, std: 0.04978, params: {'C': 0.01, 'gamma': 4},
  mean: -1.56048, std: 0.05007, params: {'C': 0.01, 'gamma': 5},
  mean: -1.57011, std: 0.05013, params: {'C': 0.01, 'gamma': 6},
  mean: -1.57508,

In [180]:
# SVR
from sklearn import svm
svr = svm.SVR(C = 10, gamma = 2)
svr.fit(df_train_var_left, Y_train)
test_pred=svr.predict(df_test_var_left)

# test_pred = np.round(test_pred)

for i in range(len(test_pred)):
    if test_pred[i] >= 5:
        test_pred[i] = 5

review_id = test['review_id']
out = pd.DataFrame()
out[0] = review_id
out[1] = test_pred
out.to_csv('out.csv', header = False, index = False)

In [200]:
# SVR tolerance rounding
file_name = 'out/RMSE_0.8624_ACC_0.4398_RFfeature_3000_SVR_round_5.csv'
data = pd.read_csv(file_name, header = None)
data = pd.DataFrame(data)
test_pred = list(data[1])

tol = 0.1
for i in range(len(test_pred)):
    if 1-tol <= test_pred[i] <= 1+tol:
        test_pred[i] = 1
    elif 2-tol <= test_pred[i] <= 2+tol:
        test_pred[i] = 2
    elif 3-tol <= test_pred[i] <= 3+tol:
        test_pred[i] = 3
    elif 4-tol <= test_pred[i] <= 4+tol:
        test_pred[i] = 4
    elif test_pred[i] >= 5-tol:
        test_pred[i] = 5
        
review_id = test['review_id']
out = pd.DataFrame()
out[0] = review_id
out[1] = test_pred
out.to_csv('out.csv', header = False, index = False)